In [ ]:
# The aim of this module is to use the Ben Graham model to score the companies

# File name convention
# R_ = raw extract 
# E_ = Entry files into enrichment layer
# C_ = enriched layer with calculation
# I = insights layer, designed for model baselines
# V_ = validation


In [3]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

I_FLAT = pd.read_pickle ('/Users/joezhou/Downloads/C_TICKER_FLAT.pkl') 


In [17]:
# Selection using ENTervative investment approach
# large company with track record, with good long standing financials

BG_ENT = I_FLAT

BG_ENT['CALC_BG_ENT_2_SALES'] = np.where(BG_ENT['totalRevenue'] > 1000000000 , 1, 0)
BG_ENT['CALC_BG_ENT_3_CURR_RATIO'] = np.where(BG_ENT['currentRatio'] > 1.5 , 1, 0)
BG_ENT['CALC_BG_ENT_4_DIV'] = np.where((BG_ENT['dividendYield'] ) > 0, 1, 0)
BG_ENT['CALC_BG_ENT_4_DIV_GTH'] = 0
# INF_SUBSET['FLAG_4_DIV_GTH'] = np.where((INF_SUBSET['fiveYearAvgDividendYield']) > 0, 1, 0)

BG_ENT['CALC_BG_ENT_5_ERN'] = np.where(BG_ENT['grossProfits'] > 0 , 1, 0)
BG_ENT['CALC_BG_ENT_6_ERN_GTH'] = np.where(BG_CONS['earningsGrowth'] > .02 , 1, 0)

# 7. Market cap < (asset - liability) * 1.5
BG_CONS['CALC_BG_CONS_7_MKT_CAP'] = np.where(BG_CONS['marketCap'] > .02 , 1, 0)
BG_CONS['CALC_BG_CONS_8_PE'] = np.where(BG_CONS['trailingPE'] < 15 , 1, 0)

BG_CONS['CALC_BG_CONS_BG_SCORE'] = BG_CONS['CALC_BG_CONS_2_SALES'] + BG_CONS['CALC_BG_CONS_3_CURR_RATIO'] + BG_CONS['CALC_BG_CONS_4_DIV']+ BG_CONS['CALC_BG_CONS_4_DIV_GTH']+ BG_CONS['CALC_BG_CONS_5_ERN']+ BG_CONS['CALC_BG_CONS_6_ERN_GTH']+ BG_CONS['CALC_BG_CONS_7_MKT_CAP']

# only keep variables used in and output from BG model
BG_CONS_VARS = BG_CONS[['TickName','totalRevenue','currentRatio','dividendYield','fiveYearAvgDividendYield','grossProfits','earningsGrowth','marketCap','trailingPE',
'CALC_BG_CONS_2_SALES','CALC_BG_CONS_3_CURR_RATIO','CALC_BG_CONS_4_DIV','CALC_BG_CONS_4_DIV_GTH','CALC_BG_CONS_5_ERN',
'CALC_BG_CONS_6_ERN_GTH','CALC_BG_CONS_7_MKT_CAP','CALC_BG_CONS_8_PE','CALC_BG_CONS_BG_SCORE'
]]

BG_CONS_SUM = BG_CONS.groupby(['CALC_BG_CONS_BG_SCORE'])['TickName'].count().reset_index()
BG_CONS_SUM.head(10)


,CALC_BG_CONS_BG_SCORE,TickName
0,0,253
1,1,176
2,2,719
3,3,691
4,4,253
5,5,155
6,6,18


In [18]:
# Selection using enterprising investment approach
# large company positioned for growth
# 

BG_ENT = I_FLAT

BG_ENT['CALC_BG_ENT_2_SALES'] = np.where(BG_ENT['revenueGrowth'] > 0 , 1, 0)
BG_ENT['CALC_BG_ENT_3_CURR_RATIO'] = np.where(BG_ENT['currentRatio'] > 1 , 1, 0)
BG_ENT['CALC_BG_ENT_4_DIV'] = 0
BG_ENT['CALC_BG_ENT_4_DIV_GTH'] = np.where((BG_ENT['fiveYearAvgDividendYield']) > 0, 1, 0)

BG_ENT['CALC_BG_ENT_5_ERN'] = np.where(BG_ENT['grossProfits'] > 0 , 1, 0)
BG_ENT['CALC_BG_ENT_6_ERN_GTH'] = np.where(BG_ENT['earningsGrowth'] > .02 , 1, 0)

# 7. Market cap < (asset - liability) * 1.5

BG_ENT['CALC_BG_ENT_7_MKT_CAP'] = np.where(BG_ENT['marketCap'] > .02 , 1, 0)
BG_ENT['CALC_BG_ENT_8_PE'] = np.where(BG_ENT['trailingPE'] < 25 , 1, 0)

BG_ENT['CALC_BG_ENT_BG_SCORE'] = BG_ENT['CALC_BG_ENT_2_SALES'] + BG_ENT['CALC_BG_ENT_3_CURR_RATIO'] + BG_ENT['CALC_BG_ENT_4_DIV']+ BG_ENT['CALC_BG_ENT_4_DIV_GTH']+ BG_ENT['CALC_BG_ENT_5_ERN']+ BG_ENT['CALC_BG_ENT_6_ERN_GTH']+ BG_ENT['CALC_BG_ENT_7_MKT_CAP']


# only keep variables used in and output from BG model
BG_ENT_VARS = BG_ENT[['TickName','CALC_BG_ENT_2_SALES','CALC_BG_ENT_3_CURR_RATIO','CALC_BG_ENT_4_DIV','CALC_BG_ENT_4_DIV_GTH','CALC_BG_ENT_5_ERN',
'CALC_BG_ENT_6_ERN_GTH','CALC_BG_ENT_7_MKT_CAP','CALC_BG_ENT_8_PE','CALC_BG_ENT_BG_SCORE'
]]

BG_ENT_SUM = BG_ENT.groupby(['CALC_BG_ENT_BG_SCORE'])['TickName'].count().reset_index()
BG_ENT_SUM.head(10)


,CALC_BG_ENT_BG_SCORE,TickName
0,0,254
1,1,133
2,2,484
3,3,591
4,4,541
5,5,164
6,6,98


In [20]:
BG_MODEL = BG_CONS_VARS.merge(BG_ENT_VARS,how='left',on='TickName')
BG_MODEL.to_pickle("/Users/joezhou/Downloads/C_BEN_G_MODEL.pkl")
BG_MODEL.to_excel('/Users/joezhou/Downloads/C_BEN_G_MODEL.xlsx', engine='xlsxwriter')  

